In [33]:
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector
from datetime import datetime


In [47]:
load_dotenv()

True

In [31]:
def query_all_table_from_mysql(table_name,db_config):
    conn = mysql.connector.connect(**db_config)    
    cursor = conn.cursor()

    query = f'SELECT * FROM {table_name}'

    cursor.execute(query)

    # Fetch the results into a pandas DataFrame
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

    conn.commit()
    cursor.close()
    conn.close()

    return df

def edited_query_from_table_mysql(db_config,sql_query):
    conn = mysql.connector.connect(**db_config)    
    cursor = conn.cursor()

    query = sql_query

    cursor.execute(query)

    # Fetch the results into a pandas DataFrame
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

    conn.commit()
    cursor.close()
    conn.close()

    return df

In [20]:
#Configurando Datos de Conexión a la DB
db_config = {
    "host": os.getenv("MYSQL_HOST"),
    "user": os.getenv("MYSQL_USER"),
    "password": os.getenv("MYSQL_PASSWORD"),
    "database": os.getenv("MYSQL_DATABASE")
}

In [21]:
# df_store_information = query_all_table_from_mysql('store_information', db_config)
# df_actual_stock = query_all_table_from_mysql('actual_stock', db_config)
# df_warehouse_stock = query_all_table_from_mysql('warehouse_stock', db_config)
df_product_raw = query_all_table_from_mysql("product_information",db_config)


In [39]:
today = datetime.now()
year_now = today.year
last_year = year_now - 1
min_date = f'{last_year}-01-01'
max_date = f'{last_year}-12-31'
df_ventas = edited_query_from_table_mysql(db_config,f'SELECT * FROM sales where `Date` between "{min_date}" and "{max_date}"')


In [44]:
df_product_raw = df_product_raw[['ProductId','cod_fami']]
df_last_year_sales = df_ventas.merge(df_product_raw, how = 'left', on = 'ProductId')
df_last_year_sales

,codigo_corto,ProductId,StoreId,TicketId,Date,Quantity,PriceTaxesExcluded,SalesPersonId,Origen,cod_fami
0,721957,199980.1001.TM,1013700,L37173777,2023-01-28,1.00,15050.42,9553,POS,251
1,1164081,206636.3480.150CM,1011000,L101372066,2023-01-28,2.00,22178.15,9306,POS,239
2,834238,209361.1000.UNI,1011000,L101372023,2023-01-28,1.00,6799.16,9318,POS,251
3,878783,210189.0005.147CM,1010100,L012853441,2023-01-28,0.80,20161.34,106868,POS,239
4,926710,218044.0005.T10,1010100,L012856067,2023-01-28,1.00,45285.71,9597,POS,248
...,...,...,...,...,...,...,...,...,...,...
2904046,1428213,269276.0005.TM,1010100,L012855777,2023-01-28,1.00,58815.13,50025,POS,250
2904047,1408663,266641.0005.TS,1010100,L012855777,2023-01-28,1.00,23523.53,50025,POS,250
2904048,1434955,270020.1100.TS,1010100,L012855777,2023-01-28,1.00,50411.76,50025,POS,250
2904049,1406592,266423.1090.T08,1010100,L012855777,2023-01-28,1.00,47052.94,50025,POS,250
